In [1]:
from __future__ import division
from __future__ import print_function
import os, sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import argparse
import random
import numpy as np
import scipy.sparse as sp
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

from torch import optim
import torch.nn.functional as F
from model import ROD_cla
from optimizer import loss_function
from utils import *
from sklearn.cluster import SpectralClustering, KMeans
from clustering_metric import clustering_metrics
from tqdm import tqdm
import time

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=200)
parser.add_argument('--hidden', type=int, default=128)
parser.add_argument('--lr', type=float, default=0.02)
parser.add_argument('--weight_decay', type=float, default=5e-4)
parser.add_argument('--dropout', type=float, default=0.8)
parser.add_argument('--batch_size', type=int, default=1000)
parser.add_argument('--dataset', type=str, default='cora')
parser.add_argument('--num_hops', type=int, default=9)
parser.add_argument('--device', type=int, default=0)
args = parser.parse_args(args=[])
print("Using {} dataset".format(args.dataset))

Using cora dataset


### 1. Data Fetching

In [ ]:
device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
adj, features, labels, idx_train, idx_val, idx_test = load_data(args.dataset)
features = F.normalize(features, 1)
labels = torch.LongTensor(labels)

adj_1st = (adj + sp.eye(adj.shape[0])).toarray()
adj_label = torch.FloatTensor(adj_1st)
neg_num = pos_num = adj_label.sum().long()

adj = aug_normalized_adjacency(adj)
adj = sparse_mx_to_torch_sparse_tensor(adj).float()

n_nodes = features.shape[0]
input_feature = [features]
for i in range(args.num_hops):
    propagated_fea = torch.spmm(adj,input_feature[-1])
    input_feature.append(propagated_fea)
feature_list = [fea.to(device) for fea in input_feature]
    
labels = labels.to(device)
adj = adj.to(device)

### 2. Data Preprocessing

In [11]:
sm_sim_mx_list = []
for i in range(args.num_hops+1):
    cur_feat = F.normalize(feature_list[i].cpu().data)
    sm_sim_mx_list.append(torch.mm(cur_feat, cur_feat.t()).reshape([-1,]))

pos_inds_list = []
neg_inds_list = []
for i in range(args.num_hops+1):
    pos_inds_list.append(np.argpartition(-sm_sim_mx_list[i], pos_num)[:pos_num])
    neg_inds_list.append(np.argpartition(sm_sim_mx_list[i], pos_num*100)[:pos_num*100])

length = len(pos_inds_list[0])
length_neg = len(neg_inds_list[0])

pos_inds_cuda_list = [torch.LongTensor(pos_inds).to(device) for pos_inds in pos_inds_list]

batch_size = args.batch_size

### 3. Node Classification

In [14]:
model = ROD_cla(nfeat=features.shape[1], nhid=args.hidden,
        nclass=labels.max().item() + 1, dropout=args.dropout, num_hops=args.num_hops).to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

t_total = time.time()
record = {}
for epoch in tqdm(range(args.epochs)):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    z_list= model(feature_list)
    
    z_ensemble = F.softmax(z_list[0].data, 1)
    for i in range(1, args.num_hops+1):
        z_ensemble += F.softmax(z_list[i].data, 1)
    z_ensemble /= args.num_hops + 1
    
    loss1 = 0.
    for i in range(args.num_hops+1):
        loss1 += F.cross_entropy(z_list[i][idx_train], labels[idx_train], reduction='mean')
    
    loss2 = 0.
    for i in range(args.num_hops+1):
        loss2 += F.kl_div(F.log_softmax(z_list[i][idx_train], dim=1), z_ensemble[idx_train], reduction='batchmean')
    
    start, end = 0, batch_size
    loss3 = 0.
    ran_head = np.random.randint(0, length_neg-length-1)
    sampled_neg_list = []
    for i in range(args.num_hops+1):
        sampled_neg_list.append(torch.LongTensor(neg_inds_list[i][np.arange(ran_head, ran_head+length)]).to(device))
    
    while end <= length:
        for i in range(args.num_hops+1):
            sampled_inds = torch.cat((pos_inds_cuda_list[i][start:end], sampled_neg_list[i][start:end]), 0)
            xind = sampled_inds // n_nodes
            yind = sampled_inds % n_nodes
            zx = torch.index_select(z_list[i], 0, xind)
            zy = torch.index_select(z_list[i], 0, yind)
            batch_label = torch.cat((torch.ones(end-start), torch.zeros(end-start)), 0).to(device)
            batch_pred = (F.normalize(zx) * F.normalize(zy)).sum(1)
            weight = torch.cat((batch_pred[:batch_size], 1-batch_pred[batch_size:]), 0).data
            loss3 += loss_function(adj_preds=batch_pred, adj_labels=batch_label, weight=weight)

        start = end
        if end < length <= end + batch_size:
            end += length - end
        else:
            end += batch_size

    loss_train = loss1 + 0.01*loss2 + 0.001*loss3
    acc_train = accuracy(z_ensemble[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
                          
    model.eval()
    z_list = model(feature_list)
    z_ensemble = z_list[0]
    for i in range(1, args.num_hops+1):
        z_ensemble += z_list[i]
    z_ensemble /= args.num_hops + 1

    acc_val = accuracy(z_ensemble[idx_val], labels[idx_val])
    acc_test = accuracy(z_ensemble[idx_test], labels[idx_test])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'acc_test: {:.4f}'.format(acc_test.item()),
          'time: {:.4f}s'.format(time.time() - t))
    record[acc_val.item()] = acc_test.item()
        
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
bit_list = sorted(record.keys())
bit_list.reverse()
for key in bit_list[:10]:
    value = record[key]
    print(key,value)

  0%|          | 1/200 [00:00<01:09,  2.86it/s]

Epoch: 0001 loss_train: 19.5118 acc_train: 0.1286 acc_val: 0.0600 acc_test: 0.0650 time: 0.3477s


  1%|          | 2/200 [00:00<01:05,  3.04it/s]

Epoch: 0002 loss_train: 19.4878 acc_train: 0.1929 acc_val: 0.2120 acc_test: 0.1930 time: 0.2807s


  2%|▏         | 3/200 [00:00<00:59,  3.32it/s]

...

Epoch: 0198 loss_train: 2.3755 acc_train: 0.9429 acc_val: 0.8020 acc_test: 0.8140 time: 0.2243s


100%|█████████▉| 199/200 [00:45<00:00,  4.41it/s]

Epoch: 0199 loss_train: 2.9148 acc_train: 0.9357 acc_val: 0.8020 acc_test: 0.8150 time: 0.2266s


100%|██████████| 200/200 [00:45<00:00,  4.38it/s]

Epoch: 0200 loss_train: 2.5731 acc_train: 0.9500 acc_val: 0.7980 acc_test: 0.8220 time: 0.2242s
Optimization Finished!
Total time elapsed: 45.6927s
0.8280000000000001 0.843
0.8260000000000001 0.842
0.8240000000000001 0.843
0.8160000000000001 0.838
0.8140000000000001 0.834
0.812 0.833
0.81 0.833
0.808 0.834
0.806 0.8290000000000001
0.804 0.8240000000000001
